In [1]:
# Python library imports: numpy, random, sklearn, pandas, etc

import warnings
warnings.filterwarnings('ignore')

import sys
import random
import numpy as np

# cross_validation is deprecated since version 0.18. This module will be removed in 0.20.
# Use sklearn.model_selection.train_test_split instead.
# from sklearn import linear_model, cross_validation, metrics, svm
from sklearn.model_selection import train_test_split
from sklearn import linear_model, metrics, svm
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

import pandas as pd
import matplotlib.pyplot as plt
import pickle
import time
%matplotlib inline

In [2]:
# Config size
TRAINING_MATRIX_RESULT_FILE_NAME = 'training_matrix_result.pickle'

In [30]:
# from date import date
from datetime import date
# # Python UDFs for our PIG script # from date import date # get hour-of-day from HHMM field @outputSchema("value: int") def get_hour(val): return int(val.zfill(4)[:2]) # this array defines the dates of holiday in 2007 and 2008
# holidays = [date(2007, 1, 1), date(2007, 1, 15), date(2007, 2, 19), date(2007, 5, 28), date(2007, 6, 7), date(2007, 7, 4), \ date(2007, 9, 3), date(2007, 10, 8), date(2007, 11, 11), date(2007, 11, 22), date(2007, 12, 25), 
#             date(2008, 1, 1), date(2008, 1, 21), date(2008, 2, 18), date(2008, 5, 22), date(2008, 5, 26), date(2008, 7, 4), \ date(2008, 9, 1), date(2008, 10, 13), date(2008, 11, 11), date(2008, 11, 27), date(2008, 12, 25) \ ] # get number of days from nearest holiday @outputSchema("days: int") def days_from_nearest_holiday(year, month, day): d = date(year, month, day) x = [(abs(d-h)).days for h in holidays] return min(x)

holiday_list = []

# Hard code holiday
holiday_1987 = [date(1987, 1, 1), date(1987, 1, 19), date(1987, 2, 16), date(1987, 5, 1), date(1987, 5, 25), date(1987, 7, 4), date(1987, 9, 7), date(1987, 10, 12), date(1987, 11, 11), date(1987, 11, 26), date(1987, 12, 25)]
holiday_1988 = [date(1988, 1, 1), date(1988, 1, 18), date(1988, 2, 15), date(1988, 5, 1), date(1988, 5, 30), date(1988, 7, 4), date(1988, 9, 5), date(1988, 10, 10), date(1988, 11, 11), date(1988, 11, 24), date(1988, 12, 25)]
holiday_1989 = [date(1989, 1, 1), date(1989, 1, 16), date(1989, 2, 20), date(1989, 5, 1), date(1989, 5, 29), date(1989, 7, 4), date(1989, 9, 4), date(1989, 10, 9), date(1989, 11, 11), date(1989, 11, 23), date(1989, 12, 25)]
holiday_1990 = [date(1990, 1, 1), date(1990, 1, 15), date(1990, 2, 19), date(1990, 5, 1), date(1990, 5, 28), date(1990, 7, 4), date(1990, 9, 3), date(1990, 10, 8), date(1990, 11, 11), date(1990, 11, 22), date(1990, 12, 25)]
holiday_1991 = [date(1991, 1, 1), date(1991, 1, 21), date(1991, 2, 18), date(1991, 5, 1), date(1991, 5, 27), date(1991, 7, 4), date(1991, 9, 2), date(1991, 10, 14), date(1991, 11, 11), date(1991, 11, 28), date(1991, 12, 25)]
holiday_1992 = [date(1992, 1, 1), date(1992, 1, 20), date(1992, 2, 17), date(1992, 5, 1), date(1992, 5, 25), date(1992, 7, 4), date(1992, 9, 7), date(1992, 10, 12), date(1992, 11, 11), date(1992, 11, 26), date(1992, 12, 25)]
holiday_1993 = [date(1993, 1, 1), date(1993, 1, 18), date(1993, 2, 15), date(1993, 5, 1), date(1993, 5, 31), date(1993, 7, 4), date(1993, 9, 6), date(1993, 10, 11), date(1993, 11, 11), date(1993, 11, 25), date(1993, 12, 25)]
holiday_1994 = [date(1994, 1, 1), date(1994, 1, 17), date(1994, 2, 21), date(1994, 5, 1), date(1994, 5, 30), date(1994, 7, 4), date(1994, 9, 5), date(1994, 10, 10), date(1994, 11, 11), date(1994, 11, 24), date(1994, 12, 25)]
holiday_1995 = [date(1995, 1, 1), date(1995, 1, 16), date(1995, 2, 20), date(1995, 5, 1), date(1995, 5, 29), date(1995, 7, 4), date(1995, 9, 4), date(1995, 10, 9), date(1995, 11, 11), date(1995, 11, 23), date(1995, 12, 25)]
holiday_1996 = [date(1996, 1, 1), date(1996, 1, 15), date(1996, 2, 19), date(1996, 5, 1), date(1996, 5, 27), date(1996, 7, 4), date(1996, 9, 2), date(1996, 10, 14), date(1996, 11, 11), date(1996, 11, 28), date(1996, 12, 25)]
holiday_1997 = [date(1997, 1, 1), date(1997, 1, 20), date(1997, 2, 17), date(1997, 5, 1), date(1997, 5, 26), date(1997, 7, 4), date(1997, 9, 1), date(1997, 10, 13), date(1997, 11, 11), date(1997, 11, 27), date(1996, 12, 25)]

holiday_list.extend(holiday_1987)
holiday_list.extend(holiday_1988)
holiday_list.extend(holiday_1989)
holiday_list.extend(holiday_1990)
holiday_list.extend(holiday_1991)
holiday_list.extend(holiday_1992)
holiday_list.extend(holiday_1993)
holiday_list.extend(holiday_1994)
holiday_list.extend(holiday_1995)
holiday_list.extend(holiday_1996)
holiday_list.extend(holiday_1997)

def days_from_nearest_holiday(year, month, day): 
    d = date(year, month, day) 
    x = [(abs(d-h)).days for h in holiday_list]
    return min(x)

In [4]:
load_training_model_st = time.time()

# Load trained model from file
result_location_path = f"./training_results/{TRAINING_MATRIX_RESULT_FILE_NAME}"
clf_rf = pickle.load(open(result_location_path, "rb"))

# Load training model time
load_training_model_et = time.time()
elapsed_time = load_training_model_et - load_training_model_st
print('Execution time:', elapsed_time, 'seconds')

Execution time: 21.52083420753479 seconds


### Import data and make prediction

In [5]:
load_data_st = time.time()
# read files
df_1997 = pd.read_csv('result/1997.csv')

df_years = [df_1997]

flight_df = pd.concat(df_years)
flight_df

# Load data time
load_data_et = time.time()
elapsed_time = load_data_et - load_data_st
print('Execution time:', elapsed_time, 'seconds')

Execution time: 11.454945087432861 seconds


In [31]:
process_hdays_st = time.time()

# Process holidays
flight_df['hdays'] = flight_df.apply(lambda row: days_from_nearest_holiday(row['year'], row['month'], row['day_of_month']), axis=1)

# elapsed time
process_hdays_et = time.time()
elapsed_time = process_hdays_et - process_hdays_st
print('Execution time:', elapsed_time, 'seconds')

Execution time: 215.31143879890442 seconds


In [7]:
def make_a_copy(dataframe):
    return dataframe.copy()

In [44]:
# Make a map dictionary of the unique_carrier series to match with the training models unique_carrier category value
unique_mapping_dict = {
    'AA': 1,
    'AS': 2,
    'CO': 3,
    'DL': 4,
    'EA': 5,
    'HP': 6,
    'ML (1)': 7,
    'NW': 8,
    'PA (1)': 9,
    'PI': 10,
    'PS': 11,
    'TW': 12,
    'UA': 13,
    'US': 14,
    'WN': 15
}
training_unique_carrier = {'unique_carrier': ['UA', 'DL', 'US', 'WN', 'CO', 'AA', 'PI', 'TW', 'NW', 'HP', 'EA',
       'AS', 'PA (1)', 'PS', 'ML (1)']}
training_unique_carrier_df = pd.DataFrame(data=training_unique_carrier)
training_unique_carrier_df['unique_carrier'].replace(unique_mapping_dict, inplace=True)
training_unique_carrier_df['unique_carrier']

0     13
1      4
2     14
3     15
4      3
5      1
6     10
7     12
8      8
9      6
10     5
11     2
12     9
13    11
14     7
Name: unique_carrier, dtype: int64

In [32]:
processing_flight_df = make_a_copy(flight_df)

In [33]:
# Select columns to reduce the training matrix side and define dimension for prediction feature
cols = ['unique_carrier', 'month', 'flight_num', 'day_of_month', 'day_of_week', 'dest', 'origin', 'dep_time', 'distance', 'dep_delay', 'hdays']
flight_df_selc = processing_flight_df[cols]
# flight_df_selc['unique_carrier'].unique()
flight_df_selc["unique_carrier"].values.reshape(-1, 1)


array([['WN'],
       ['WN'],
       ['WN'],
       ...,
       ['AA'],
       ['AA'],
       ['AA']], dtype=object)

In [34]:
flight_df_selc["unique_carrier"].unique()

array(['WN', 'DL', 'HP', 'CO', 'US', 'UA', 'AA', 'NW', 'TW', 'AS'],
      dtype=object)

In [35]:
# process dimension to map with the correct format of the training models
flight_df_selc['dep_delay'] = (flight_df_selc['dep_delay'] > 10) *1
flight_df_selc['unique_carrier'].replace(unique_mapping_dict, inplace=True)

# categ_cols = ['unique_carrier', 'flight_num', 'dest', 'origin', 'hdays']
# For now the hdays is not applied yet. Need to figure out (or ignore for now)
categ_cols = ['flight_num', 'dest', 'origin']

for item  in categ_cols:
    flight_df_selc[item] = flight_df_selc[item].astype('category').cat.codes + 1


In [36]:
flight_df_selc["unique_carrier"].unique()

array([15,  4,  6,  3, 14, 13,  1,  8, 12,  2])

In [37]:
flight_df_selc["hdays"].unique()

array([ 8,  9, 10, 11,  6,  0,  1,  2,  4,  5,  7,  3, 14, 13, 12, 15, 16,
       17, 18, 19, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 35, 36,
       34, 20, 27])

In [38]:
# Divide test and train matrix
train, test, y_train, y_test = train_test_split(flight_df_selc.drop(['dep_delay'], axis=1), flight_df_selc["dep_delay"],
                                                random_state=10, train_size=1)



In [39]:
test["unique_carrier"].values.reshape(-1, 1)

array([[15],
       [ 8],
       [15],
       ...,
       [ 8],
       [ 6],
       [ 1]])

In [45]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
enc.fit(training_unique_carrier_df["unique_carrier"].values.reshape(-1, 1))  

# enc.feature_indices_
airline_onehot_test = enc.transform(test["unique_carrier"].values.reshape(-1, 1)).toarray()

In [46]:
air_oh_test_df = pd.DataFrame(airline_onehot_test, columns = ["A1","A2","A3","A4","A5","A6","A7",
                                                         "A8","A9","A10", "A11", "A12", "A13", "A14", "A15"])

In [47]:
test2  = pd.concat([test.reset_index(),air_oh_test_df.reset_index()],axis=1).drop(["index","month","day_of_month","day_of_week","unique_carrier","flight_num","dest","origin", "hdays"],axis=1)

test2

,dep_time,distance,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15
0,2025,328,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1933,453,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1715,457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1138,178,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,701,641,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5301993,600,361,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5301994,1520,175,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5301995,1036,408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5301996,947,843,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Evaluate on 1997 test set

In [48]:
predict_st = time.time()

# Evaluate on test set
pr = clf_rf.predict(test2)

# print results
cm = confusion_matrix(y_test, pr)
print("Confusion matrix")
print(pd.DataFrame(cm))
report_svm = precision_recall_fscore_support(list(y_test), list(pr), average='micro')
print("\nprecision = %0.2f, recall = %0.2f, F1 = %0.2f, accuracy = %0.2f\n" % \
        (report_svm[0], report_svm[1], report_svm[2], accuracy_score(list(y_test), list(pr))))


# save time
predict_et = time.time()
elapsed_time = predict_et - predict_st
print('Execution time:', elapsed_time, 'seconds')

Confusion matrix
         0       1
0  3628517  641119
1   484286  548076

precision = 0.79, recall = 0.79, F1 = 0.79, accuracy = 0.79

Execution time: 95.15947103500366 seconds


In [49]:
y_test2 = pd.DataFrame({"y":y_test.reset_index(drop=True)})
predict2 = pd.DataFrame({"yhat":pr})
x_test2 = pd.DataFrame(test2)
result = x_test2.join(y_test2)
predicted_result = result.join(predict2)
predicted_result

,dep_time,distance,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,y,yhat
0,2025,328,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1
1,1933,453,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
2,1715,457,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0
3,1138,178,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
4,701,641,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5301993,600,361,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0
5301994,1520,175,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
5301995,1036,408,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
5301996,947,843,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [50]:
predicted_result[predicted_result['y'] != predicted_result['yhat']]

,dep_time,distance,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,y,yhat
0,2025,328,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0,1
3,1138,178,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
5,601,576,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
9,845,328,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0
13,1831,861,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5301978,2350,2066,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
5301981,2025,256,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0
5301988,1844,719,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,1
5301989,857,83,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0


In [51]:
# Predict real data
predict_st = time.time()

# Evaluate on test set
pr = clf_rf.predict(test2)

# save time
predict_et = time.time()
elapsed_time = predict_et - predict_st
print('Execution time:', elapsed_time, 'seconds')

Execution time: 70.70775604248047 seconds


In [53]:
# Showing result
pr

array([1, 0, 0, ..., 0, 0, 0])